<div style="width:90%;
            margin:10px;
            padding:8px;
            border:2px solid #FF0000;
            border-radius:20px;
            font-weight:bold;
            font-size:2.5em;
            text-align:center;">
The Space Shuttle Reentry Problem Using JuMP with different solvers
</div>


<div style="width:90%;
            margin:9px;
            font-size:1em;">
            
The aim here is to compare the results of different solvers for the same problem: the space shuttle reentry problem using JuMP. For the integration method, we will use the trapezoidal rule. 

1. [Comparing Linear Solvers](#ls)
2. [Comparing Non Linear Solvers](#nls)
3. [Results](#res)


</div>

### Importing the Model

In [1]:
import Pkg
Pkg.activate("..")
include("./utils.jl");
using OptimalControlProblems
using Plots
using MKL
using JuMP, Ipopt
import HSL_jll
using DataFrames, BenchmarkTools
using PrettyTables, Colors

nh = 200;
results = DataFrame( :method => String[], 
                    :total_time => String[],
                    :objective_value => Float64[],
                    :iterations => Int64[]);


  Activating project at `c:\Users\hajje\Dropbox\PC\Desktop\Stage\COTS`


## 1. Comparing Linear Solvers : MUMPS, HSL_MA57 and HSL_MA27  <a id="ls"></a>

#### 1. MUMPS

In [2]:
model = space_shuttle(JuMPBackend();integration_rule="trapezoidal")
set_silent(model)
set_optimizer(model,Ipopt.Optimizer)
set_optimizer_attribute(model, "sb", "yes")
set_optimizer_attribute(model,"linear_solver", "mumps")
b = @benchmark optimize!(model) evals=1 samples=1
push!(results,["Ipopt + MUMPS",
                prettytime(b.times[1]),
                round(objective_value(model) |> rad2deg;digits=7),
                solution_summary(model).barrier_iterations
                ]);

#### 2. HSL_MA57

In [3]:
model = space_shuttle(JuMPBackend();integration_rule="trapezoidal")
set_silent(model)
set_optimizer(model,Ipopt.Optimizer)
set_attribute(model, "hsllib", HSL_jll.libhsl_path)
set_attribute(model, "linear_solver", "ma57")
b = @benchmark optimize!(model) evals=1 samples=1
push!(results,["Ipopt + HSL_MA57",
                prettytime(b.times[1]),
                round(objective_value(model) |> rad2deg;digits=7),
                solution_summary(model).barrier_iterations
                ]);

#### 3. HSL_MA27

In [4]:
model = space_shuttle(JuMPBackend();integration_rule="trapezoidal")
set_silent(model)
set_optimizer(model,Ipopt.Optimizer)
set_attribute(model, "hsllib", HSL_jll.libhsl_path)
set_attribute(model, "linear_solver", "ma27")
b = @benchmark optimize!(model) evals=1 samples=1
push!(results,["Ipopt + HSL_MA27",
                prettytime(b.times[1]),
                round(objective_value(model) |> rad2deg;digits=7),
                solution_summary(model).barrier_iterations
                ]);

## 1. Results <a id="res"></a>

In [6]:
# Define the custom display function
hl_v1 = Highlighter(
           (results, i, j) -> (j == 3) && (round(results[i, j],digits=4) != 34.1412),
           crayon"red bold"
       );
hl_v2 = Highlighter(
            (results, i, j) -> (j == 3) && (results[i, j] != 34.1411822),
            crayon"red"
        );
header = ["Method", "Total Time", "Objective Value", "Iterations"];
# Apply the custom display function to the :objective_value column
pretty_table(
    results;
    formatters    = ft_printf("%.7f", 2:3),
    header        = header,
    header_crayon = crayon"yellow bold",
    highlighters  = (hl_v1, hl_v2),
    tf            = tf_unicode_rounded
)

╭──────────────────┬──────────────┬─────────────────┬────────────╮
│           Method │   Total Time │ Objective Value │ Iterations │
├──────────────────┼──────────────┼─────────────────┼────────────┤
│    Ipopt + MUMPS │ 11.4542308 s │      34.1411822 │        124 │
│ Ipopt + HSL_MA57 │  9.4139305 s │      34.1411822 │        124 │
│ Ipopt + HSL_MA27 │ 11.6510529 s │      34.1411822 │        124 │
╰──────────────────┴──────────────┴─────────────────┴────────────╯
